In [61]:
import numpy as np
import random
import time

In [62]:
class Cell():
    def __init__(self, cell_id, x, y):
        self.cell_id = cell_id
        self.x = x
        self.y = y
        self.is_occupied = False
        self.next_cell = None
 

    def get_root(self):
        cell = self
        while cell.next_cell:
            cell = cell.next_cell
        return cell

    
    def set_root(self, other):
        self.next_cell = other
    
    
    def occupy_cell(self):
        self.is_occupied = True
    
    
    def compress_path(self, root):
        if self.next_cell:
            cell = self
            while cell.next_cell:
                tmp = cell.next_cell
                cell.next_cell = root
                cell = tmp
            cell.next_cell = root
        else:
            self.next_cell = root
    
    
    def get_neighbours(self, L):
        xl = self.x - 1 if self.x > 0 else (L - 1)
        yu = self.y + 1 if self.y < (L - 1) else 0
        xr = self.x + 1 if self.x < (L - 1) else 0
        yd = self.y - 1 if self.y > 0 else (L - 1)
        return [(xl, y), (x, yu), (xr, y), (x, yd)]
    
    
    def __eq__(self, other):
        return (self.cell_id == other.cell_id)    
    
    
    def __ne__(self, other):
        return (self.cell_id != other.cell_id)    
    
    

In [63]:
class Cluster():
    dimension = 0
    def __init__(self, root, L):
        self.root = root
        self.cells = [root]
        self.xmin = self.xmax = root.x
        self.ymin = self.ymax = root.y
        Cluster.dimension = L
        
    def get_cluster_id(self):
        return self.root.cell_id
        
    def get_cluster_size(self):
        return len(self.cells)
        
    def __add__(self, other):
        self.cells += other.cells
        self.xmax = max(self.xmax, other.xmax)
        self.xmin = min(self.xmin, other.xmin)
        self.ymax = max(self.ymax, other.ymax)
        self.ymin = min(self.ymin, other.ymin)
        
    def does_percolate(self, L):
        return ((self.xmax - self.xmin) > (Cluster.L - 1)) or ((self.ymax - self.ymin) > (Cluster.L - 1))
        
    
    def __eq__(self, other):
        return (self.root.cell_id == other.root.cell_id)
    
    def __gt__(self, other):
        return (self.get_cluster_size() > other.get_cluster_size())
    
    def _lt__(self, other):
        return (self.get_cluster_size() < other.get_cluster_size())
            

In [64]:
# some helper functions

def plot_lattice(lattice, N):
    tmp = np.full(lattice.shape, 0, dtype='i')
    for i in range(N):
        x = int(i % L)
        y = int((i - x) / L)
        if lattice[x][y].is_occupied:
            tmp[x][y] = lattice[x][y].get_root().cell_id
    print(tmp)

def print_cluster(clustersize):
    for cluster, size in clustersize.items():
        if size > 0:
            print(cluster, " ", size)


def print_clusters(clusters):
    for key, cluster in clusters.items():
        print(key, cluster.get_cluster_size())

In [65]:
L = 10
N = L * L

# create a lattice and fill it with 'empty' cells
lattice = np.ndarray([L, L], dtype = type(Cell))
coordinates = []
for i in range(N):
    x = int(i % L)
    y = int((i - x) / L)
    lattice[x][y] = Cell(i + 1, x, y)
    coordinates.append((x, y))
    
    
# shuffle coordinates to randomly select cells to be filled:
np.random.shuffle(coordinates)

In [67]:
start = time.time()

# keep track of cluster sizes
sizes = [0 for i in range(N + 1)]
ids = [i for i in range(N + 1)]
cluster_size = dict(zip(ids, sizes))

#cluster
clusters = {}

for (x, y) in coordinates:
        
    lattice[x][y].occupy_cell() 
    
    cluster_size[lattice[x][y].get_root().cell_id] = 1
    
    #cluster 
    clusters[lattice[x][y].get_root().cell_id] = Cluster(lattice[x][y], L)
    print(clusters[lattice[x][y].get_root().cell_id])
    
    #check for neighbouring cells:
    nn_coordinates = lattice[x][y].get_neighbours(L)
    
    for (nx, ny) in nn_coordinates:
        if lattice[nx][ny].is_occupied:
           
            r1 = lattice[x][y].get_root()
            size1 = cluster_size[r1.cell_id]
            
            r2 = lattice[nx][ny].get_root()
            size2 = cluster_size[r2.cell_id]
    
            if r1 != r2:
                print_clusters(clusters)
                print("found n. cluster for cell/root ",id1,r1.cell_id,"with cell/root",id2,r2.cell_id)
                if size1 < size2:
                    lattice[x][y].compress_path(r2)
                    cluster_size[r1.cell_id] = 0
                    cluster_size[r2.cell_id] += size1
                    
                    #cluster
                    if clusters[r2.cell_id]:
                        clusters[r2.cell_id] += clusters[r1.cell_id]
                    else:
                        clusters[r2.cell_id] = clusters[r1.cell_id]
                    clusters.pop(r1.cell_id)
                    
                    print('size',cluster_size[r2.cell_id], clusters[r2.cell_id].get_cluster_size())
                    
                else: 
                    lattice[nx][ny].compress_path(r1)
                    cluster_size[r1.cell_id] += size2
                    cluster_size[r2.cell_id] = 0 
                    
                    #cluster
                    if clusters[r1.cell_id]:
                        clusters[r1.cell_id] += clusters[r2.cell_id]
                    else:
                        clusters[r1.cell_id] = clusters[r2.cell_id]
                    clusters.pop(r2.cell_id)
                print_clusters(clusters)
            else:
                if size1 < size2:
                    #cluster
                    clusters[r2.cell_id] += clusters[r1.cell_id]
                else:
                    #cluster
                    clusters[r1.cell_id] += clusters[r2.cell_id]
                print("Percolation", clusters[r1.cell_id].does_percolate())
    print()
    plot_lattice(lattice, N)
    print_cluster(cluster_size)
stop = time.time()
print(stop - start)


[[ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 42  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0 28 28  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]]
42   1


AttributeError: 'NoneType' object has no attribute 'does_percolate'